In [5]:
import torch
from torch import nn

In [40]:
float("-inf")

-inf

In [2]:
[
    i
    for i in range(10)
]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [31]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 seq_len:int,
                 embed_dim:int) -> None:
        """
        Parameters:
            seq_len: length of input sequence. aka number of words in a sentence
            embed_dim: dimension of embedding. Ex: 512
        """
        super().__init__()

        self.seq_len = seq_len
        self.embed_dim = embed_dim

        pos_encoding = torch.zeros(size=(self.seq_len, self.embed_dim), dtype=torch.float32)
        angle_numerator = torch.arange(self.seq_len, dtype=torch.float32).unsqueeze(1)
        angle_denominator = torch.pow(10000.0, (2 * (torch.arange(self.embed_dim, dtype=torch.float32) // 2)) / self.embed_dim).unsqueeze(0)
        angles = angle_numerator / angle_denominator

        pos_encoding[:, 0::2] = torch.sin(angles[:, 0::2])
        pos_encoding[:, 1::2] = torch.cos(angles[:, 1::2])
        self.register_buffer("pos_encoding", pos_encoding)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        z = self.pos_encoding[:, :x.size(1)]
        return x + z.requires_grad_(False)
        

In [29]:
x = torch.randint(1, 10, [10, 5, 8])
x.size(-2), x.size(-1), x[0]

(5,
 8,
 tensor([[7, 6, 8, 6, 1, 1, 4, 9],
         [8, 3, 1, 5, 2, 6, 6, 1],
         [8, 6, 1, 1, 7, 2, 4, 1],
         [2, 3, 6, 5, 1, 9, 8, 5],
         [3, 1, 8, 4, 2, 7, 6, 4]]))

In [32]:
pe = PositionalEncoding(5, 8)

In [39]:
pe(x[0])

tensor([[ 7.0000,  7.0000,  8.0000,  7.0000,  1.0000,  2.0000,  4.0000, 10.0000],
        [ 8.8415,  3.5403,  1.0998,  5.9950,  2.0100,  6.9999,  6.0010,  2.0000],
        [ 8.9093,  5.5839,  1.1987,  1.9801,  7.0200,  2.9998,  4.0020,  2.0000],
        [ 2.1411,  2.0100,  6.2955,  5.9553,  1.0300,  9.9995,  8.0030,  6.0000],
        [ 2.2432,  0.3464,  8.3894,  4.9211,  2.0400,  7.9992,  6.0040,  5.0000]])

In [4]:
def _generate_square_mask(seq_len:int):
    mask = (torch.triu(torch.ones(seq_len, seq_len)) == 1).transpose(0, 1)
    return mask


def _gen_mask(seq_len:int):
    mask = (1-torch.triu(torch.ones(seq_len, seq_len), diagonal=1)).bool()
    return mask

### Tạo Mask

In [5]:
mask1 = (torch.triu(torch.ones(5, 5)) == 1).transpose(0, 1) # Ấn Độ viết
mask1 = mask1.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
mask1

NameError: name 'mask' is not defined

In [92]:
(1 - torch.triu(torch.ones(5, 5), diagonal=1)).bool() # Ấn Độ sống ở Ý viết

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])

In [72]:
mask = torch.tril(torch.ones(5, 5)) # Tui sống ở Việt Name viết
mask

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [41]:
mask2 = torch.tril(torch.ones(5, 5)).expand(4, 1, 5, 5)
mask2

tensor([[[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]]])

In [42]:
attn_scores = torch.randint(1, 10, (4, 5, 5)).float()
attn_scores

tensor([[[1., 2., 1., 5., 6.],
         [8., 5., 4., 5., 8.],
         [5., 3., 4., 6., 4.],
         [6., 5., 3., 4., 9.],
         [8., 2., 1., 1., 7.]],

        [[2., 7., 3., 3., 4.],
         [4., 2., 5., 4., 7.],
         [2., 4., 4., 9., 4.],
         [7., 8., 5., 7., 3.],
         [4., 4., 9., 6., 7.]],

        [[2., 2., 3., 6., 2.],
         [7., 4., 8., 5., 5.],
         [4., 8., 2., 1., 5.],
         [7., 8., 7., 2., 3.],
         [7., 6., 3., 5., 4.]],

        [[1., 4., 3., 9., 2.],
         [4., 6., 1., 6., 7.],
         [9., 9., 9., 2., 3.],
         [9., 8., 3., 6., 2.],
         [9., 2., 7., 5., 8.]]])

In [68]:
attn_scores.masked_fill(mask==0, float('-inf'))

tensor([[[1., -inf, -inf, -inf, -inf],
         [8., 5., -inf, -inf, -inf],
         [5., 3., 4., -inf, -inf],
         [6., 5., 3., 4., -inf],
         [8., 2., 1., 1., 7.]],

        [[2., -inf, -inf, -inf, -inf],
         [4., 2., -inf, -inf, -inf],
         [2., 4., 4., -inf, -inf],
         [7., 8., 5., 7., -inf],
         [4., 4., 9., 6., 7.]],

        [[2., -inf, -inf, -inf, -inf],
         [7., 4., -inf, -inf, -inf],
         [4., 8., 2., -inf, -inf],
         [7., 8., 7., 2., -inf],
         [7., 6., 3., 5., 4.]],

        [[1., -inf, -inf, -inf, -inf],
         [4., 6., -inf, -inf, -inf],
         [9., 9., 9., -inf, -inf],
         [9., 8., 3., 6., -inf],
         [9., 2., 7., 5., 8.]]])

In [2]:
mask

NameError: name 'mask' is not defined

In [21]:
torch.tril(torch.ones(5, 5)) == 0

tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])

In [20]:
torch.triu(torch.ones(5, 5), diagonal=1).bool()

tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])

In [56]:
import random 
mylist = [1,2,3,4,5,6,7,8,9,10,11,12]
# random.shuffle(mylist)
n = len(mylist)
a = int(n * 0.8)
train = mylist[:a]
test = mylist[a:]
a, train, test

(9, [1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12])

In [75]:
def causal_mask(size):
    # mask = torch.triu(torch.ones((size, size)), diagonal=1).type(torch.int)
    # return mask == 0
    mask = torch.tril(torch.ones(1, size, size))
    return mask.bool()

In [19]:
pad_token = torch.tensor([2])
pad_token

tensor([2])

In [20]:
decoder_input = torch.tensor([10, 12, 14, 16, 1, 2, 2, 2])
decoder_input

tensor([10, 12, 14, 16,  1,  2,  2,  2])

In [76]:
mask2 = causal_mask(decoder_input.shape[0])
mask2

tensor([[[ True, False, False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True,  True,  True,  True]]])

In [59]:
mask = causal_mask(decoder_input.shape[0])
mask

tensor([[ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True]])

In [48]:
(decoder_input != pad_token).int()

tensor([1, 1, 1, 1, 1, 0, 0, 0], dtype=torch.int32)

In [77]:
(decoder_input != pad_token).unsqueeze(0).int() & mask2

tensor([[[1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0]]], dtype=torch.int32)

In [78]:
print(f"{'='*10}Dataset{'='*10}")

==========Dataset==========


In [2]:
import json, string
from pyvi import ViTokenizer
from underthesea import word_tokenize, sent_tokenize

In [10]:
seq = [
    ["loài phong", "acer glabrum", "kích thước", "to lớn"],
    ["hogan's", "alley", "cộng đồng", "da", "đen"],
    ["khu phố", "strathcona", "hạt nhân", "cộng đồng", "do thái", "thành phố"]
]

In [12]:
txt = json.dumps(seq, ensure_ascii=False).encode('utf-8')
txt.decode()

'[["loài phong", "acer glabrum", "kích thước", "to lớn"], ["hogan\'s", "alley", "cộng đồng", "da", "đen"], ["khu phố", "strathcona", "hạt nhân", "cộng đồng", "do thái", "thành phố"]]'

In [14]:
a = json.loads(txt.decode())
a

[['loài phong', 'acer glabrum', 'kích thước', 'to lớn'],
 ["hogan's", 'alley', 'cộng đồng', 'da', 'đen'],
 ['khu phố', 'strathcona', 'hạt nhân', 'cộng đồng', 'do thái', 'thành phố']]

In [4]:
txt = 'Thông tin chi tiết về chương trình và các đóng góp của đối tác cùng khách hàng được FPT Long Châu công bố chính thức tại: https://nhathuoclongchau.com.vn/vi-cong-dong/chien-binh-nhi.'

In [57]:
txt.lower()

'ngay bên cạnh dự án này là khu du lịch hòn ngọc á châu ven biển đà nẵng có diện tích 12ha, vốn đầu tư 4.800 tỷ đồng, được cấp chứng nhận vào năm 2009 do công ty cp hòn ngọc á châu làm chủ đầu tư!. theo quy hoạch, dự án gồm cụm resort, biệt thự, nhà hàng, khách sạn 5 sao 18 tầng với vốn đầu tư 20 triệu usd. tuy nhiên, do gặp các vấn đề liên quan đến thủ tục pháp lý, dự án đã dừng triển khai nhiều năm qua... @#cả chục dãy dài biệt thự xây xong phần thô rồi bỏ mặc cỏ dại phủ kín trông như những căn biệt thự ma.'

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
for r in string.punctuation:
    txt = txt.replace(r, '')
txt

'Thông tin chi tiết về chương trình và các đóng góp của đối tác cùng khách hàng được FPT Long Châu công bố chính thức tại httpsnhathuoclongchaucomvnvicongdongchienbinhnhi'

In [58]:
sent_tokenize(txt.lower())

['ngay bên cạnh dự án này là khu du lịch hòn ngọc á châu ven biển đà nẵng có diện tích 12ha, vốn đầu tư 4.800 tỷ đồng, được cấp chứng nhận vào năm 2009 do công ty cp hòn ngọc á châu làm chủ đầu tư!.',
 'theo quy hoạch, dự án gồm cụm resort, biệt thự, nhà hàng, khách sạn 5 sao 18 tầng với vốn đầu tư 20 triệu usd.',
 'tuy nhiên, do gặp các vấn đề liên quan đến thủ tục pháp lý, dự án đã dừng triển khai nhiều năm qua... @#cả chục dãy dài biệt thự xây xong phần thô rồi bỏ mặc cỏ dại phủ kín trông như những căn biệt thự ma.']

In [94]:
stra = "dự án đã dừng triển khai .. nhiều &năm qua#.. dfdgl ldfgjld fdgldjgd ."

In [95]:
a = word_tokenize(stra)
a.extend(['11', '55'])
a

['dự án',
 'đã',
 'dừng',
 'triển khai',
 '..',
 'nhiều',
 '&',
 'năm',
 'qua',
 '#',
 '..',
 'dfdgl',
 'ldfgjld',
 'fdgldjgd',
 '.',
 '11',
 '55']

In [100]:
txt2 = "- Ba là, về điều hành, kiểm soát giá xăng dầu: do giá xăng dầu chịu tác động mạnh từ căng thẳng Nga - Ukraina, cũng như có ảnh hưởng lớn đối với nền kinh tế Việt Nam, TS. Cấn Văn Lực và Nhóm tác giả Viện Đào tạo và Nghiên cứu BIDV kiến nghị: (i) Bộ Công thương phối hợp cùng các cơ quan liên quan có giải pháp tổng thể đảm bảo nguồn cung xăng dầu (gồm cả trong nước và nhập khẩu); điều tiết hệ thống phân phối, tránh đứt đoạn nguồn cung, ảnh hưởng tới đời sống người dân và hoạt động sản xuất, kinh doanh của doanh nghiệp; đề xuất tăng tần suất điều chỉnh giá xăng dầu (thay vì 10 ngày như hiện nay); (ii) Chính phủ chỉ đạo rà soát, đánh giá và đề xuất phương án phù hợp đối với việc điều hành Quỹ bình ổn giá xăng dầu (cân nhắc cả phương án có nên duy trì hay không, giải pháp thay thế…); (iii) Về lâu dài, có chiến lược tăng tính tự cường, nhất là phát triển năng lượng tái tạo, tăng năng lực dự trữ và năng lực phân tích, dự báo trên cơ sở thông tin, dữ liệu và khoa học hơn."

In [107]:
special_pattern = {
    '...': '. ',
    '...v.v.': '. ',
    '... v.v': '. ',
    'v.v.': '. ',
    'v.v': '. ',
    '..': '. ',
    '....': '. ',
    ':': '. ',
    ';': '. ',
    '(i)': '',
    '(ii)': '',
    '(iii)': '',
    '(iv)': '',
    '(v)': '',
    '(vi)': '',
    '(vii)': '',
    '(viii)': '',
    '(ix)': '',
    '(x)': ''
}
for k, v in special_pattern.items():
    txt2 = txt2.replace(k, v)
sent_tokenize(txt2)

['- Ba là, về điều hành, kiểm soát giá xăng dầu.',
 'do giá xăng dầu chịu tác động mạnh từ căng thẳng Nga - Ukraina, cũng như có ảnh hưởng lớn đối với nền kinh tế Việt Nam, TS. Cấn Văn Lực và Nhóm tác giả Viện Đào tạo và Nghiên cứu BIDV kiến nghị.',
 'Bộ Công thương phối hợp cùng các cơ quan liên quan có giải pháp tổng thể đảm bảo nguồn cung xăng dầu (gồm cả trong nước và nhập khẩu).',
 'điều tiết hệ thống phân phối, tránh đứt đoạn nguồn cung, ảnh hưởng tới đời sống người dân và hoạt động sản xuất, kinh doanh của doanh nghiệp.',
 'đề xuất tăng tần suất điều chỉnh giá xăng dầu (thay vì 10 ngày như hiện nay).',
 'Chính phủ chỉ đạo rà soát, đánh giá và đề xuất phương án phù hợp đối với việc điều hành Quỹ bình ổn giá xăng dầu (cân nhắc cả phương án có nên duy trì hay không, giải pháp thay thế…).',
 'Về lâu dài, có chiến lược tăng tính tự cường, nhất là phát triển năng lượng tái tạo, tăng năng lực dự trữ và năng lực phân tích, dự báo trên cơ sở thông tin, dữ liệu và khoa học hơn.']

In [3]:
word_tokenize("Hồi đầu năm, công ty cũng công bố hoàn thành vòng gọi vốn Series D, với sự tham gia của các nhà đầu tư đang là cổ đông hiện hữu bao gồm Warburg Pincus, Affirma Capital và Tybourne Capital Management.")

['Hồi',
 'đầu',
 'năm',
 ',',
 'công ty',
 'cũng',
 'công bố',
 'hoàn thành',
 'vòng',
 'gọi',
 'vốn',
 'Series D',
 ',',
 'với',
 'sự',
 'tham gia',
 'của',
 'các',
 'nhà đầu tư',
 'đang',
 'là',
 'cổ đông',
 'hiện hữu',
 'bao gồm',
 'Warburg Pincus',
 ',',
 'Affirma Capital',
 'và',
 'Tybourne',
 'Capital Management',
 '.']

In [45]:
def origin_sentence(src_text:str) -> str:
    patterns = {
        '_': ' ',
        ' ,': ',',
        ' .': '.',
        ' ?': '?',
        ' :': ':'
    }
    for k, v in patterns.items():
        text = text.replace(k, v)

    # Capitalize the first char of the word after dot sign
    i = text.find('. ')
    if i != -1:
        while True:
            i += 2
            if i < len(text):
                text = text[:i] + text[i].capitalize() + text[(i + 1):]
            i = text.find('. ', i)
            if i == -1:
                break
    text = text[0].capitalize() + text[1:]
    return text

In [46]:
txt1 = "để được tư_vấn , hỗ_trợ tìm_hiểu chi_tiết các sản_phẩm tiềm_năng , khách_hàng vui_lòng liên_hệ việt_nhân bình_dương ."
txt2 = "phú_quốc – những con_số biết nói . để được tư_vấn , hỗ_trợ tìm_hiểu chi_tiết các sản_phẩm tiềm_năng . khách_hàng vui_lòng liên_hệ việt_nhân bình_dương . số_liệu từ sở du_lịch tỉnh kiên_giang cho biết , trong 2 tháng đầu năm 2024 , lượng khách du_lịch đến phú_quốc tăng đột_biến ."

In [49]:
print(txt1)
origin_sentence(txt1)

để được tư_vấn , hỗ_trợ tìm_hiểu chi_tiết các sản_phẩm tiềm_năng , khách_hàng vui_lòng liên_hệ việt_nhân bình_dương .


'Để được tư vấn, hỗ trợ tìm hiểu chi tiết các sản phẩm tiềm năng, khách hàng vui lòng liên hệ việt nhân bình dương.'

In [1]:
"Hello World".count(" ")

1

In [4]:
pad = torch.tensor([2], dtype=torch.int64)
pad

tensor([2])

In [7]:
torch.tensor([2] * 10, dtype=torch.int64)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [9]:
A = [1,5,9,0,4,67,12]

In [2]:
max(A)

67

In [4]:
max([i for i in A])

67

In [15]:
tokens = torch.full((5, 10), 2, dtype=torch.int64)
tokens[0, : len(A)] = torch.tensor(A)

In [70]:
mask_padding = (tokens != 2).int()
mask_padding

tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)

In [71]:
mask = torch.tril(torch.ones(1, 10, 10)).bool()
mask

tensor([[[ True, False, False, False, False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False, False, False],
         [ True,  True,  True,  True, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]]])

In [74]:
torch.stack([i & mask for i in mask_padding], dim=0)  

tensor([[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]],


        [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]],


        [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 

In [73]:
torch.cat([i & mask for i in mask_padding], dim=0)  

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0,